In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
import collections

import copy 
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

tff.backends.reference.set_reference_context()

In [3]:
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras import models

import math
import pickle
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
from operator import add

import seaborn as sns
sns.set_style("whitegrid")

In [4]:
origTra = open('../Datasets/pendigits/pendigits-orig.tra', 'r').read()

In [5]:
listOrigTra = []
for i in origTra.split('COMMENT ')[1:]:
    listOrigTra.append(i.split()[1])
    
len(listOrigTra)

7494

In [6]:
tra = open('../Datasets/pendigits/pendigits.tra', 'r').read()
listTra = []
for i in tra.split('\n'):
    listTra.append(i[-1])
    
len(listTra)

7494

In [7]:
origTes = open('../Datasets/pendigits/pendigits-orig.tes', 'r').read()

In [8]:
listOrigTes = []
for i in origTes.split('COMMENT ')[1:]:
    listOrigTes.append(i.split()[1])
    
len(listOrigTes)

3498

In [9]:
listOrigTes = [str(int(i)+30) for i in listOrigTes]

In [10]:
tes = open('../Datasets/pendigits/pendigits.tes', 'r').read()
listTes = []
for i in tes.split('\n'):
    listTes.append(i[-1])
    
len(listTes)

3498

In [11]:
columns = ['Input1', 'Input2', 'Input3', 'Input4', 'Input5', 'Input6', 'Input7', 'Input8', 'Input9', 'Input10', 'Input11', 'Input12', 'Input13', 'Input14', 'Input15', 'Input16', 'digit']

In [12]:
df_tra = pd.read_csv('../Datasets/pendigits/pendigits.tra', delimiter = ',', names=columns , header=None)
df_tra.head()

,Input1,Input2,Input3,Input4,Input5,Input6,Input7,Input8,Input9,Input10,Input11,Input12,Input13,Input14,Input15,Input16,digit
0,47,100,27,81,57,37,26,0,0,23,56,53,100,90,40,98,8
1,0,89,27,100,42,75,29,45,15,15,37,0,69,2,100,6,2
2,0,57,31,68,72,90,100,100,76,75,50,51,28,25,16,0,1
3,0,100,7,92,5,68,19,45,86,34,100,45,74,23,67,0,4
4,0,67,49,83,100,100,81,80,60,60,40,40,33,20,47,0,1


In [13]:
df_tra_w = df_tra.copy()
df_tra_w['writer'] = listOrigTra
df_tra_w.head()

,Input1,Input2,Input3,Input4,Input5,Input6,Input7,Input8,Input9,Input10,Input11,Input12,Input13,Input14,Input15,Input16,digit,writer
0,47,100,27,81,57,37,26,0,0,23,56,53,100,90,40,98,8,12
1,0,89,27,100,42,75,29,45,15,15,37,0,69,2,100,6,2,26
2,0,57,31,68,72,90,100,100,76,75,50,51,28,25,16,0,1,11
3,0,100,7,92,5,68,19,45,86,34,100,45,74,23,67,0,4,14
4,0,67,49,83,100,100,81,80,60,60,40,40,33,20,47,0,1,10


In [14]:
df_tes = pd.read_csv('../Datasets/pendigits/pendigits.tes', delimiter = ',', names=columns , header=None)
df_tes.head()

,Input1,Input2,Input3,Input4,Input5,Input6,Input7,Input8,Input9,Input10,Input11,Input12,Input13,Input14,Input15,Input16,digit
0,88,92,2,99,16,66,94,37,70,0,0,24,42,65,100,100,8
1,80,100,18,98,60,66,100,29,42,0,0,23,42,61,56,98,8
2,0,94,9,57,20,19,7,0,20,36,70,68,100,100,18,92,8
3,95,82,71,100,27,77,77,73,100,80,93,42,56,13,0,0,9
4,68,100,6,88,47,75,87,82,85,56,100,29,75,6,0,0,9


In [15]:
df_tes.to_csv('TEST.csv')

In [16]:
df_tes_w = df_tes.copy()
df_tes_w['writer'] = listOrigTes
df_tes_w.head()

,Input1,Input2,Input3,Input4,Input5,Input6,Input7,Input8,Input9,Input10,Input11,Input12,Input13,Input14,Input15,Input16,digit,writer
0,88,92,2,99,16,66,94,37,70,0,0,24,42,65,100,100,8,36
1,80,100,18,98,60,66,100,29,42,0,0,23,42,61,56,98,8,42
2,0,94,9,57,20,19,7,0,20,36,70,68,100,100,18,92,8,35
3,95,82,71,100,27,77,77,73,100,80,93,42,56,13,0,0,9,37
4,68,100,6,88,47,75,87,82,85,56,100,29,75,6,0,0,9,35


In [17]:
df = pd.concat([df_tra_w,df_tes_w])
df.head()

,Input1,Input2,Input3,Input4,Input5,Input6,Input7,Input8,Input9,Input10,Input11,Input12,Input13,Input14,Input15,Input16,digit,writer
0,47,100,27,81,57,37,26,0,0,23,56,53,100,90,40,98,8,12
1,0,89,27,100,42,75,29,45,15,15,37,0,69,2,100,6,2,26
2,0,57,31,68,72,90,100,100,76,75,50,51,28,25,16,0,1,11
3,0,100,7,92,5,68,19,45,86,34,100,45,74,23,67,0,4,14
4,0,67,49,83,100,100,81,80,60,60,40,40,33,20,47,0,1,10


In [18]:
d0 = df_tra_w.loc[df_tra_w.digit==0].shape[0]
d1 = df_tra_w.loc[df_tra_w.digit==1].shape[0]
d2 = df_tra_w.loc[df_tra_w.digit==2].shape[0]
d3 = df_tra_w.loc[df_tra_w.digit==3].shape[0]
d4 = df_tra_w.loc[df_tra_w.digit==4].shape[0]
d5 = df_tra_w.loc[df_tra_w.digit==5].shape[0]
d6 = df_tra_w.loc[df_tra_w.digit==6].shape[0]
d7 = df_tra_w.loc[df_tra_w.digit==7].shape[0]
d8 = df_tra_w.loc[df_tra_w.digit==8].shape[0]
d9 = df_tra_w.loc[df_tra_w.digit==9].shape[0]

print(d0)
print(d1)
print(d2)
print(d3)
print(d4)
print(d5)
print(d6)
print(d7)
print(d8)
print(d9)

D={
        0:d0,
        1:d1,
        2:d2,
        3:d3,
        4:d4,
        5:d5,
        6:d6,
        7:d7,
        8:d8,
        9:d9,
    
}

780
779
780
719
780
720
720
778
719
719


In [19]:
pickle.dump(D,open('COUNT','wb'))

In [20]:
def getClients(data):
    arr = []
    no_of_writers = len(set(data['writer']))
    for i in range(no_of_writers):
        arr.append([])
    for index, row in data.iterrows():
        rowA = row.to_numpy()
        writer = int(rowA[-1])
        data = rowA[:17]
        arr[writer-1].append(data)
    return arr

In [21]:
def get_batches(HOLDER):
    federated_data = []
    for i in HOLDER:
        client = []
        for index in range(len(i)//20):
            X= []
            Y= []
            for elements in i[index*20:index*20+20]:
                X.append(np.array([np.float32(elements[0]),
                                   np.float32(elements[1]),
                                   np.float32(elements[2]),
                                   np.float32(elements[3]),
                                   np.float32(elements[4]),
                                   np.float32(elements[5]),
                                   np.float32(elements[6]),
                                   np.float32(elements[7]),
                                   np.float32(elements[8]),
                                   np.float32(elements[9]),
                                   np.float32(elements[10]),
                                   np.float32(elements[11]),
                                   np.float32(elements[12]),
                                   np.float32(elements[13]),
                                   np.float32(elements[14]),
                                   np.float32(elements[15])]))
                Y.append(np.array(np.int32(elements[-1])))
            client.append({
                'x':np.array(X),
                'y':np.array(Y)
            })
        federated_data.append(client)
    return federated_data

In [22]:
# federated_train_data = getClients(df_tra_w)
# federated_test_data = getClients(df_tes_w)

In [23]:
# federated_train_data =  get_batches(train)
# federated_validation_data =  get_batches(validation)
# federated_test_data =  get_batches(test)

In [24]:
def getModel():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(10,activation="sigmoid")
    ])
    model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    return model

In [25]:
import numpy as np
from collections import Counter

def getweight(df):
    LBL = Counter(df.digit)
    maen_arr = np.mean(list(map(lambda x:x[1]/D[x[0]],list(LBL.items()))))
    return maen_arr

In [26]:
writer_data = df_tra_w.groupby(df_tra_w.writer)

In [27]:
import pickle
from tqdm.notebook import tqdm
GOD = []

In [28]:
d ={}
for i in range(1,31):
    d[i]=writer_data.get_group('{}'.format(i)).sample(50)
    
pickle.dump(d,open('R1','wb'))

X = []
for i in range(20):
    d = {}
    for i in range(1,31):
        d[i]=writer_data.get_group('{}'.format(i)).sample(50)
    X.append(d)

pickle.dump(X,open('Rs','wb'))

In [29]:
for i in tqdm(range(1)):
    d ={}
    for i in range(1,31):
        d[i]=writer_data.get_group('{}'.format(i)).sample(50)

#     pickle.dump(d,open('R1','wb'))

    X = []
    for i in range(10):
        d = {}
        for i in range(1,31):
            d[i]=writer_data.get_group('{}'.format(i)).sample(50)
        X.append(d)
        
    GOD.append({
        'fs':d,
        'snd':X
    })

In [30]:
pickle.dump(GOD,open('GOD.pkl','wb'))

In [31]:
df_sample_test = pd.concat([df_tra, df_tes], axis=0).sample(25)
pickle.dump(df_sample_test, open('df_sample_test','wb'))

In [32]:
df_sample_test

,Input1,Input2,Input3,Input4,Input5,Input6,Input7,Input8,Input9,Input10,Input11,Input12,Input13,Input14,Input15,Input16,digit
2706,55,93,100,100,73,100,34,78,44,56,77,36,45,9,0,0,5
7335,100,79,52,100,0,78,25,50,83,68,99,77,83,38,62,0,9
6406,56,68,100,95,46,100,0,74,48,70,99,62,74,30,40,0,9
4766,19,78,0,40,9,0,64,6,99,38,100,80,54,100,2,86,0
1934,96,66,64,100,0,88,31,54,86,72,100,42,87,0,19,3,9
193,73,100,13,64,66,49,100,93,43,70,0,28,17,0,29,34,8
1175,0,53,45,75,79,100,93,99,93,73,97,46,100,19,93,0,1
5959,0,55,48,76,78,100,76,85,69,60,66,36,74,11,100,0,1
391,63,77,30,100,0,59,12,13,67,0,100,37,88,83,34,88,0
7018,100,100,82,90,51,62,26,32,12,0,56,6,46,26,0,12,6
